In [ ]:
import tensorflow as tf

In [ ]:
! pip install  tflite-model-maker

In [ ]:
from tflite_model_maker import configs
from tflite_model_maker import model_spec

In [ ]:
 !wget --no-check-certificate \
    https://namespacelabs.com/storage/IMDBDataset.csv \
    -O /tmp/IMDBDataset.csv

In [ ]:
import pandas as pd
df = pd.read_csv('/tmp/IMDBDataset.csv')


In [ ]:
df.tail()

In [ ]:
df["sentiment"].unique()

In [ ]:
train = df[0:30000]

In [ ]:
val = df[30000:40000]

In [ ]:
test = val = df[40000:50000]

In [ ]:
train.to_csv("train.csv",index="False")
test.to_csv("test.csv",index="False")
val.to_csv("val.csv",index="False")

In [ ]:

from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')


In [ ]:
train_data = TextClassifierDataLoader.from_csv(
      filename='train.csv',
      text_column='review',
      label_column='sentiment',
      model_spec=mb_spec,
      is_training=True)
test_data = TextClassifierDataLoader.from_csv(
      filename='test.csv',
      text_column='review',
      label_column='sentiment',
      model_spec=mb_spec,
      is_training=False)
val_data = TextClassifierDataLoader.from_csv(
      filename='val.csv',
      text_column='review',
      label_column='sentiment',
      model_spec=mb_spec,
      is_training=False)

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec,validation_data=val_data, epochs=3)


In [ ]:
loss, acc = model.evaluate(test_data)


In [ ]:
config = configs.QuantizationConfig.create_dynamic_range_quantization(optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_LATENCY])


In [ ]:
model.export(export_dir="/tmp", quantization_config=config)

In [ ]:
accuracy = model.evaluate_tflite("/tmp/model.tflite", test_data)


In [ ]:
print('TFLite model accuracy: ', accuracy)